# Introduction

A Snowflake Semantic View is a  type of schema-level object in Snowflake that defines a business-friendly data model directly on your tables and views. Then, it can be used by AI and Humans to query data based on business needs.

## What is included in the Semantic View Definition?

* Logical Tables: These typically correspond to business entities (like CUSTOMER or ORDERS) and reference your underlying physical tables or views.

* Relationships: These explicitly define how the logical tables connect to each other (e.g., how the CUSTOMER table links to the ORDERS table via a foreign key).

* Dimensions: These are the business-friendly attributes used to group, filter, or provide context to your data (e.g., Customer_Region, Product_Category, Order_Date).

* Metrics: These are business-friendly calculations or aggregations that represent Key Performance Indicators (KPIs) (e.g., Total_Sales, Average_Order_Value, Employee_Count). They use underlying data fields, sometimes called Facts, to perform the calculation.

## How to use the Semantic View 

* Go to the **AI & ML** and search Cortex Analytics.
* Use special Syntax Query.




### Part 1: Create / Use Data to create meaningfull model

In this part you will use publicly availabile snowflake Share to create a data. 

In [ ]:
USE SCHEMA TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL;

In [ ]:
-- Use public share to create data
CREATE DATABASE IF NOT EXISTS SNOWFLAKE_SAMPLE_DATA FROM SHARE SFC_SAMPLES.SAMPLE_DATA;

In [ ]:
-- Create or replace views for the tables from SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL
CREATE OR REPLACE VIEW CUSTOMER AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.CUSTOMER;

CREATE OR REPLACE VIEW CUSTOMER_DEMOGRAPHICS AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.CUSTOMER_DEMOGRAPHICS;

CREATE OR REPLACE VIEW DATE_DIM AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.DATE_DIM;

CREATE OR REPLACE VIEW ITEM AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.ITEM;

CREATE OR REPLACE VIEW STORE AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.STORE;

CREATE OR REPLACE VIEW STORE_SALES AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.STORE_SALES;

-- Query the Data
SHOW VIEWS;
SELECT * FROM CUSTOMER LIMIT 20;

### Part 2: Prepare Semantic Model/Createa Semantic View
In this part you will get a yaml file from github repo and create a semantic view. 

### Check YAML file
You have already downloaded yaml file from github repository in setup.sql. You will check content inside yaml file. 

In [ ]:
-- Check
ls @YAML;

In [ ]:
import os
import yaml
import streamlit as st
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session

def read_yaml_from_stage(session: Session,
                         stage_path: str,
                         local_dir: str = "/tmp") -> dict:
    """
    Retrieve the YAML file from the stage to local, load it, and return its contents as a dict.

    :param session: Snowpark session
    :param stage_path: File path on the stage (e.g. "@MY_STAGE/configs/my_config.yaml")
    :param local_dir: Local directory to save the file (a writable area in the workspace)
    :return: YAML contents parsed into a dict
    """
    # → Fetch locally
    get_results = session.file.get(stage_path, local_dir)
    # get_results is a list containing info about the downloaded files
    if not get_results:
        raise FileNotFoundError(f"Failed to get file from stage: {stage_path}")
    # Normally get returns a list. Take the first result’s local file path
    local_path = get_results[0].file
    # Combine local_dir + the file name (if it’s not an absolute path)
    full_local_path = os.path.join(local_dir, local_path) if not os.path.isabs(local_path) else local_path

    # Load the YAML
    with open(full_local_path, 'r', encoding='utf-8') as f:
        yaml_data = yaml.safe_load(f)

    return yaml_data

session = get_active_session()
stage_yaml = "@TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.YAML/TPCDS_SEMANTIC_VIEW.yaml"
try:
    config = read_yaml_from_stage(session, stage_yaml, local_dir="/tmp")
    st.write(config)
except Exception as e:
    st.error("Error reading YAML:", e)

### (Optional) Create Semantic View
You are allowed to create a semantic view object via SQL query below.

In [ ]:
USE SCHEMA TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL;
CREATE OR REPLACE SEMANTIC VIEW TPCDS_SEMANTIC_VIEW_SM
	tables (
		CUSTOMER primary key (C_CUSTOMER_SK),
		DATE as DATE_DIM primary key (D_DATE_SK),
		DEMO as CUSTOMER_DEMOGRAPHICS primary key (CD_DEMO_SK),
		ITEM primary key (I_ITEM_SK),
		STORE primary key (S_STORE_SK),
		STORESALES as STORE_SALES
        primary key (SS_SOLD_DATE_SK,SS_CDEMO_SK,SS_ITEM_SK,SS_STORE_SK,SS_CUSTOMER_SK)
	)
	relationships (
		SALESTOCUSTOMER as STORESALES(SS_CUSTOMER_SK) references CUSTOMER(C_CUSTOMER_SK),
		SALESTODATE as STORESALES(SS_SOLD_DATE_SK) references DATE(D_DATE_SK),
		SALESTODEMO as STORESALES(SS_CDEMO_SK) references DEMO(CD_DEMO_SK),
		SALESTOITEM as STORESALES(SS_ITEM_SK) references ITEM(I_ITEM_SK),
		SALETOSTORE as STORESALES(SS_STORE_SK) references STORE(S_STORE_SK)
	)
	facts (
		ITEM.COST as i_wholesale_cost,
		ITEM.PRICE as i_current_price,
		STORE.TAX_RATE as S_TAX_PRECENTAGE,
        STORESALES.SALES_QUANTITY as SS_QUANTITY
	)
	dimensions (
		CUSTOMER.BIRTHYEAR as C_BIRTH_YEAR,
		CUSTOMER.COUNTRY as C_BIRTH_COUNTRY,
		CUSTOMER.C_CUSTOMER_SK as c_customer_sk,
		DATE.DATE as D_DATE,
		DATE.D_DATE_SK as d_date_sk,
		DATE.MONTH as D_MOY,
		DATE.WEEK as D_WEEK_SEQ,
		DATE.YEAR as D_YEAR,
		DEMO.CD_DEMO_SK as cd_demo_sk,
		DEMO.CREDIT_RATING as CD_CREDIT_RATING,
		DEMO.MARITAL_STATUS as CD_MARITAL_STATUS,
		ITEM.BRAND as I_BRAND,
		ITEM.CATEGORY as I_CATEGORY,
		ITEM.CLASS as I_CLASS,
		ITEM.I_ITEM_SK as i_item_sk,
		STORE.MARKET as S_MARKET_ID,
		STORE.SQUAREFOOTAGE as S_FLOOR_SPACE,
		STORE.STATE as S_STATE,
		STORE.STORECOUNTRY as S_COUNTRY,
		STORE.S_STORE_SK as s_store_sk,
		STORESALES.SS_CDEMO_SK as ss_cdemo_sk,
		STORESALES.SS_CUSTOMER_SK as ss_customer_sk,
		STORESALES.SS_ITEM_SK as ss_item_sk,
		STORESALES.SS_SOLD_DATE_SK as ss_sold_date_sk,
		STORESALES.SS_STORE_SK as ss_store_sk
	)
	metrics (
		STORESALES.TOTALCOST as SUM(item.cost),
		STORESALES.TOTALSALESPRICE as SUM(SS_SALES_PRICE),
		STORESALES.TOTALSALESQUANTITY as SUM(SS_QUANTITY)
            WITH SYNONYMS = ( 'total sales quantity', 'total sales amount')
	)
;


In [ ]:
DESC SEMANTIC VIEW TPCDS_SEMANTIC_VIEW_SM;

## Part3 Convert Semantic view to YAML file/YAML file to Semantic View
In this part, you will learn how to convert semantic view to semantic model and semantic model to semantic view. Snowflake provide two functions to convert as well as Snowsight UI.

### Method 1: Using Snowsight UI
Snowsight allows you to convert semantic view and model without coding. 
1. Sign in to Snowsight
2. Navigate to AI & ML » Cortex Analyst
3. Select "Create new" » "Upload YAML file"
4. Select your YAML file to upload
5. Select "Convert and save"


### Method 2: Using SQL functions

### YAML to Semantic View

Official Doc Link: [system_create_semantic_view_from_yaml](https://docs.snowflake.com/en/sql-reference/stored-procedures/system_create_semantic_view_from_yaml)

In [ ]:
-- Example. 
-- Warning: A error happen once this cell executed.
CALL SYSTEM$CREATE_SEMANTIC_VIEW_FROM_YAML(
  'my_db.my_schema',
  $$
  name: TPCH_REV_ANALYSIS
  description: Semantic view for revenue analysis
  tables:
    - name: CUSTOMERS
      description: Main table for customer data
      base_table:
        database: SNOWFLAKE_SAMPLE_DATA
        schema: TPCH_SF1
        table: CUSTOMER
      primary_key:
        columns:
          - C_CUSTKEY
      dimensions:
        - name: CUSTOMER_NAME
          synonyms:
            - customer name
          description: Name of the customer
          expr: customers.c_name
          data_type: VARCHAR(25)
        - name: C_CUSTKEY
          expr: C_CUSTKEY
          data_type: VARCHAR(134217728)
      metrics:
        - name: CUSTOMER_COUNT
          description: Count of number of customers
          expr: COUNT(c_custkey)
    - name: LINE_ITEMS
      description: Line items in orders
      base_table:
        database: SNOWFLAKE_SAMPLE_DATA
        schema: TPCH_SF1
        table: LINEITEM
      primary_key:
        columns:
          - L_ORDERKEY
          - L_LINENUMBER
      dimensions:
        - name: L_ORDERKEY
          expr: L_ORDERKEY
          data_type: VARCHAR(134217728)
        - name: L_LINENUMBER
          expr: L_LINENUMBER
          data_type: VARCHAR(134217728)
      facts:
        - name: DISCOUNTED_PRICE
          description: Extended price after discount
          expr: l_extendedprice * (1 - l_discount)
          data_type: "NUMBER(25,4)"
        - name: LINE_ITEM_ID
          expr: "CONCAT(l_orderkey, '-', l_linenumber)"
          data_type: VARCHAR(134217728)
    - name: ORDERS
      synonyms:
        - sales orders
      description: All orders table for the sales domain
      base_table:
        database: SNOWFLAKE_SAMPLE_DATA
        schema: TPCH_SF1
        table: ORDERS
      primary_key:
        columns:
          - O_ORDERKEY
      dimensions:
        - name: ORDER_DATE
          description: Date when the order was placed
          expr: o_orderdate
          data_type: DATE
        - name: ORDER_YEAR
          description: Year when the order was placed
          expr: YEAR(o_orderdate)
          data_type: "NUMBER(4,0)"
        - name: O_ORDERKEY
          expr: O_ORDERKEY
          data_type: VARCHAR(134217728)
        - name: O_CUSTKEY
          expr: O_CUSTKEY
          data_type: VARCHAR(134217728)
      facts:
        - name: COUNT_LINE_ITEMS
          expr: COUNT(line_items.line_item_id)
          data_type: "NUMBER(18,0)"
      metrics:
        - name: AVERAGE_LINE_ITEMS_PER_ORDER
          description: Average number of line items per order
          expr: AVG(orders.count_line_items)
        - name: ORDER_AVERAGE_VALUE
          description: Average order value across all orders
          expr: AVG(orders.o_totalprice)
  relationships:
    - name: LINE_ITEM_TO_ORDERS
      left_table: LINE_ITEMS
      right_table: ORDERS
      relationship_columns:
        - left_column: L_ORDERKEY
          right_column: O_ORDERKEY
      relationship_type: many_to_one
    - name: ORDERS_TO_CUSTOMERS
      left_table: ORDERS
      right_table: CUSTOMERS
      relationship_columns:
        - left_column: O_CUSTKEY
          right_column: C_CUSTKEY
      relationship_type: many_to_one
  $$,
TRUE);

**CREATE_SEMANTIC_VIEW_FROM_YAML** is not able to directly read YAML file stored in stage. So, you will create stored procedure including some logics to extract data inside yaml and call CREATE_SEMANTIC_VIEW_FROM_YAML function as the next cell.

In [ ]:
CREATE OR REPLACE PROCEDURE CREATE_SEMANTIC_VIEW_FROM_STAGE_YAML(
    TARGET_SCHEMA STRING,          -- e.g. 'MY_DB.MY_SCHEMA' (must be fully-qualified)
    STAGE_FILE_URL STRING,         -- e.g. '@MY_DB.MY_SCHEMA.MY_STAGE/path/model.yaml'
    VERIFY_ONLY BOOLEAN DEFAULT FALSE
)
RETURNS STRING
LANGUAGE JAVASCRIPT
EXECUTE AS CALLER
AS
$$
function exec(sql, binds) {
  var s = snowflake.createStatement({ sqlText: sql, binds: binds || [] });
  return s.execute();
}

// Split '<DB>.<SCHEMA>'
function splitSchema(fqSchema) {
  var parts = (fqSchema || '').split('.');
  if (parts.length !== 2) {
    throw new Error("TARGET_SCHEMA must be fully qualified as 'DB.SCHEMA': " + fqSchema);
  }
  return { db: parts[0], schema: parts[1] };
}

// Extract the semantic view name from YAML (first top-level 'name:' line).
// Handles simple quoted or unquoted names. If not found, throws.
function extractViewNameFromYaml(yamlText) {
  if (!yamlText) throw new Error("YAML text is empty.");
  var re = /^[ \t]*name[ \t]*:[ \t]*(?:"([^"]+)"|'([^']+)'|([^\r\n#]+))/mi;
  var m = yamlText.match(re);
  if (!m) {
    throw new Error("Could not find a top-level 'name:' in the YAML specification.");
  }
  var raw = (m[1] || m[2] || m[3] || "").trim();
  // Remove inline trailing comments if present.
  raw = raw.replace(/[ \t]+#[^\r\n]*$/, "").trim();
  return raw;
}

var ctx = splitSchema(TARGET_SCHEMA);

// 1) Create a temporary file format to read YAML as a single column ($1).
var ff_name = 'FF_YAML_' + Date.now();
var ff_fqn  = ctx.db + '.' + ctx.schema + '.' + ff_name;

exec(
  'CREATE TEMPORARY FILE FORMAT ' + ff_fqn +
  ' TYPE = CSV FIELD_DELIMITER = NONE RECORD_DELIMITER = NONE'
);

// 2) Read YAML from stage using the named file format.
var yamlText;
try {
  var rs = exec(
    'SELECT $1 AS YAML_TEXT FROM ' + STAGE_FILE_URL + ' (FILE_FORMAT => ?) LIMIT 1',
    [ff_fqn]
  );
  if (!rs.next()) {
    throw new Error('Failed to read YAML from stage: ' + STAGE_FILE_URL);
  }
  yamlText = String(rs.getColumnValue(1));
} finally {
  // Best-effort cleanup (TEMP also drops at session end).
  try { exec('DROP FILE FORMAT IF EXISTS ' + ff_fqn); } catch (e) {}
}

// 3) Determine the target semantic view FQN from YAML.
var viewNameFromYaml = extractViewNameFromYaml(yamlText);
// Preserve quotes if the YAML used a quoted identifier with spaces.
var semanticViewFqn = ctx.db + '.' + ctx.schema + '.' + viewNameFromYaml;

// 4) Create or verify the semantic view from YAML.
//    Note: the system proc returns a status string; we return the FQN explicitly.
var callRs = exec(
  'CALL SYSTEM$CREATE_SEMANTIC_VIEW_FROM_YAML(?, ?, ?)',
  [ (ctx.db + '.' + ctx.schema), yamlText, VERIFY_ONLY ]
);
callRs.next(); // status (not used further)

if (VERIFY_ONLY) {
  return 'VERIFIED: ' + semanticViewFqn;
} else {
  return 'CREATED: ' + semanticViewFqn;
}
$$;

In [ ]:
CALL CREATE_SEMANTIC_VIEW_FROM_STAGE_YAML(
  'TKO_APJ_SEMANTIC_VIEW.PUBLIC',
  '@TKO_APJ_SEMANTIC_VIEW.PUBLIC.YAML/TPCDS_SEMANTIC_VIEW.yaml',
  FALSE  -- 
);

In [ ]:
-- Check
DESC SEMANTIC VIEW TKO_APJ_SEMANTIC_VIEW.PUBLIC.TPCDS_SEMANTIC_VIEW_SM

### Semantic View to YAML file

Official Doc Link: [system_read_yaml_from_semantic_view](https://docs.snowflake.com/en/sql-reference/functions/system_read_yaml_from_semantic_view)

In [ ]:
SELECT SYSTEM$READ_YAML_FROM_SEMANTIC_VIEW('TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.TPCDS_SEMANTIC_VIEW_SM') as yaml_format

In [ ]:
-- Save it in internal stage. 
COPY INTO @TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.YAML/TPCDS_SEMANTIC_VIEW_2.yaml
FROM (
  SELECT SYSTEM$READ_YAML_FROM_SEMANTIC_VIEW('TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.TPCDS_SEMANTIC_VIEW_SM')
)
FILE_FORMAT = (
  TYPE = CSV
  FIELD_DELIMITER = NONE 
  RECORD_DELIMITER = NONE
  FIELD_OPTIONALLY_ENCLOSED_BY = NONE
  COMPRESSION = NONE
)
SINGLE = TRUE
OVERWRITE = TRUE;

In [ ]:
ls @YAML

### Part 4: How you can use a Semantic View

In [ ]:
SELECT * FROM SEMANTIC_VIEW
( 
 TPCDS_SEMANTIC_VIEW_SM
    DIMENSIONS 
            Item.Brand,
            Item.Category,            
            Date.Year,
            Date.Month,
            Store.State
    METRICS 
        StoreSales.TotalSalesQuantity
    WHERE
        Date.Year = '2002' AND Date.Month = '12' AND Store.State ='TX' AND Item.Category = 'Books'
) 
ORDER BY TotalSalesQuantity DESC LIMIT 10;

## Part 5: Integrate Semantic View with Cortex Agents/Snowflake Intellifence. 
In this part, you will learn how to integrate semantic view with Cortex Agents and some tips to improve accuracy/performance of Semantic view. 

1. Sign in to Snowsight
2. Navigate to AI & ML » Cortex Agents
3. ??

TODO: Put some tips to improve accuracy of its on top of CI/CD pipeline


## Part6 Omni free trial 

TODO: Put some commnets to how to create Omni free trial and so on. 

## (Optionaly) share semantic view to another snowflake account 

TODO: Put some sql commands to how to share it with another snowflake account 